# Step 4 - Agents and Agentic Systems

This notebook demonstrates the implementation of agentic systems using CrewAI, a framework for creating multi-agent AI systems. It covers:

1. Setting up the development environment by installing necessary libraries.
2. Creating a news publishing agentic system with three agents:
   - Planner Agent: Plans the content of the article
   - Writer Agent: Writes the content based on the planner's outline
   - Editor Agent: Edits the blog post to align with the organization's style

3. Building a customer support agentic system with two agents:
   - Support Agent: Provides answers to user queries
   - Support Quality Assurance Agent: Verifies the accuracy of the support agent's responses

The notebook demonstrates how to:
- Define agents with specific roles, goals, and backstories
- Create tasks for each agent
- Set up a crew of agents to work together
- Use tools like web scraping to enhance agent capabilities
- Execute the agentic system and review the output

This practical implementation showcases the power of multi-agent systems in automating complex tasks such as content creation and customer support.

#### Note: This notebook is based on the course from DeepLearning.AI called [Multi AI Agent Systems With CrewAI](https://www.deeplearning.ai/short-courses/multi-ai-agent-systems-with-crewai/)

## Install Relevant Libraries

In [2]:
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29 openai

  Using cached crewai-0.28.8-py3-none-any.whl.metadata (13 kB)
  Using cached crewai_tools-0.1.6-py3-none-any.whl.metadata (4.6 kB)
  Using cached langchain_community-0.0.29-py3-none-any.whl.metadata (8.3 kB)
  Using cached openai-1.37.1-py3-none-any.whl.metadata (22 kB)
  Using cached instructor-0.5.2-py3-none-any.whl.metadata (10 kB)
  Using cached langchain-0.1.20-py3-none-any.whl.metadata (13 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl.metadata (21 kB)
  Using cached chromadb-0.4.24-py3-none-any.whl.metadata (7.3 kB)
  Using cached lancedb-0.5.7-py3-none-any.whl.metadata (17 kB)
  Using cached pytube-15.0.0-py3-none-any.whl.metadata (5.0 kB)
  Using cached aiohttp-3.9.5-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (7.5 kB)
  Using cached dataclasses_json-0.6.7-py3-none-any.whl.metadata (25 kB)
  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
  Using cached aiosignal-1.3.1-py3-none-any.whl.metadata (4.0 kB)
  Using cached fro

## Import Relevant Libraries

In [37]:
import os
import openai
from crewai import Agent, Task, Crew
from crewai_tools import SerperDevTool, ScrapeWebsiteTool, WebsiteSearchTool

# Warning control
import warnings
warnings.filterwarnings('ignore')

# OPENAI Credentials
os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"
openai.api_key = os.getenv("OPENAI_API_KEY")
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4-turbo'

## Building A News Publishing Agentic System

The system comprises three (3) Agents:
- Planner Agent: Responsible for planning out the content of the Article to be published
- Writer Agent: Responsible for writing out the content of the Article based on the outline provided by the Planner Agent
- Editor Agent: Responsible for editing the blog post to align with the writing style of the organization.

In [11]:
# Create the Planner Agent
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

In [12]:
# Create the writer Agent
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

In [13]:
# Create the editor agent
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

In [14]:
# Create the task for the Planner Agent
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

In [15]:
# Create the Task for the writer Agent
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

In [16]:
# Create the Task for the Editor Agent
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

In [17]:
# Create the crew with all the agents
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

In [19]:
%%time
# Start the crew with a blog topic of your choice
result = crew.kickoff(inputs={"topic": "Agriculture in Ghana"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Agriculture in Ghana.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
Thought: I now can give a great answer

Final Answer: 

**Content Title:** "Green Innovations and Challenges: The Current Landscape of Agriculture in Ghana"

**Target Audience:** 
- Investors interested in African agriculture
- Policy makers focusing on food security and economic development
- Agricultural professionals and farmers
- Students and researchers in agricultural studies
- NGOs focused on sustainable agriculture

**Audience Interests and Pain Points:**
- Understanding viable investment opportunities in Ghana's agricultural sector
- Insight into s

## Conclusion

Observe from the above output how the agents collaborated to execute the task. The planning agent laid out the plan for the blog, the writer agent wrote it out and the editor made the final touches.

Now you have a Publication Firm

In [23]:
# View the output from the agents
from IPython.display import Markdown
Markdown(result)

```markdown
# Green Innovations and Challenges: The Current Landscape of Agriculture in Ghana

## Introduction: A Pillar of National Prosperity
Agriculture in Ghana is not merely a traditional activity but a cornerstone of the nation's economy, contributing significantly to its GDP and providing employment to a large part of the population. As we explore this sector's vibrant landscape, we recognize its potential in shaping Ghana's economic future.

## Latest Trends in Ghanaian Agriculture: Embracing Modernity
In recent years, Ghana has seen a transformative shift towards sustainable and organic farming practices, driven by the dual need for environmental conservation and enhanced food security. The integration of cutting-edge agricultural technologies such as drones and Internet of Things (IoT) applications is revolutionizing farming practices, moving away from traditional methods to more efficient and sustainable systems like agroforestry and mixed farming.

## Key Players in the Agricultural Sector: The Backbone of Growth
The agricultural framework of Ghana is supported by key stakeholders that include governmental bodies like the Ministry of Food and Agriculture, alongside major local agribusinesses such as Wienco Ghana Limited and Agricare Limited. International partners and investors, coupled with NGOs like Heifer International and USAID, also play crucial roles in driving agricultural innovation and sustainability in the region.

## Noteworthy News and Developments: Policies and Progress
The Ghanaian government's recent policies have significantly impacted agricultural growth, focusing on enhancing productivity and sustainability. Success stories from local farming initiatives provide scalable models that underscore the potential of strategic policy implementation. Moreover, the emphasis on agricultural education and training programs is preparing a new generation of farmers equipped with knowledge and modern techniques.

## Challenges Facing Agriculture in Ghana: Overcoming Obstacles
Despite remarkable progress, Ghana's agricultural sector faces several challenges. Climate change continues to pose a significant threat, disrupting traditional farming seasons and outputs. Issues such as inadequate market access, complex land ownership rights, and limited access to finance further hinder growth. Addressing these issues requires concerted efforts and innovative strategies.

## Opportunities for Growth and Investment: A Fertile Ground
The agricultural landscape in Ghana presents lucrative opportunities for growth and investment, particularly in emerging markets like organic products and export-oriented crops. Technological innovations, particularly in precision agriculture, offer promising frontiers for investors. Partnerships between the government and private sectors could further catalyze the sector's development.

## Call to Action: Invest, Support, Engage
We invite investors to explore the vibrant opportunities in Ghana’s agricultural technology and policies that promote sustainable practices. Engaging with local agricultural projects can lead to substantial gains for both investors and the community. Your involvement can drive the change towards a more sustainable and prosperous agricultural future in Ghana.

## Conclusion: Seeding the Future
The agricultural sector in Ghana stands at a crossroads of tradition and innovation. By capitalizing on the opportunities for sustainable practices and investment, we can support not only economic growth but also environmental sustainability. Let us commit to nurturing this vital sector, ensuring it continues to thrive and benefit future generations.
```

# Customer Support Agents

This is a customer support agentic system to help customers with issues surrounding their use of the langchain library.

This system comprises of two main agents:
- Support Agent: Responsible for providing the right answers to a user's query.
- Support Quality Assurance Agent: Responsible for verifying that the answer's from the Support Agent are correct

In [28]:
# Create the support agent
support_agent = Agent(
    role="Senior Support Representative",
	goal="Be the most friendly and helpful "
        "support representative in your team",
	backstory=(
		"You work at Langchain (https://www.langchain.com/) and "
        " are now working on providing "
		"support to {customer}, a super important customer "
        " for your company."
		"You need to make sure that you provide the best support!"
		"Make sure to provide full complete answers, "
        " and make no assumptions."
	),
	allow_delegation=False,
	verbose=True
)

In [29]:
# Create the quality assurance agent
support_quality_assurance_agent = Agent(
	role="Support Quality Assurance Specialist",
	goal="Get recognition for providing the "
    "best support quality assurance in your team",
	backstory=(
		"You work at Langchain (https://www.langchain.com/) and "
        "are now working with your team "
		"on a request from {customer} ensuring that "
        "the support representative is "
		"providing the best support possible.\n"
		"You need to make sure that the support representative "
        "is providing full"
		"complete answers, and make no assumptions."
	),
	verbose=True
)

In [30]:
# Create a tool for scraping the given url
docs_scrape_tool = ScrapeWebsiteTool(
    website_url="https://python.langchain.com/v0.2/docs/introduction/"
)

In [32]:
# Create the task for the support agent
inquiry_resolution = Task(
    description=(
        "{customer} just reached out with a super important ask:\n"
	    "{inquiry}\n\n"
        "{person} from {customer} is the one that reached out. "
		"Make sure to use everything you know "
        "to provide the best support possible."
		"You must strive to provide a complete "
        "and accurate response to the customer's inquiry."
    ),
    expected_output=(
	    "A detailed, informative response to the "
        "customer's inquiry that addresses "
        "all aspects of their question.\n"
        "The response should include references "
        "to everything you used to find the answer, "
        "including external data or solutions. "
        "Ensure the answer is complete, "
		"leaving no questions unanswered, and maintain a helpful and friendly "
		"tone throughout."
    ),
	tools=[docs_scrape_tool],
    agent=support_agent,
)

In [33]:
# Create the task for the quality assurance agent
quality_assurance_review = Task(
    description=(
        "Review the response drafted by the Senior Support Representative for {customer}'s inquiry. "
        "Ensure that the answer is comprehensive, accurate, and adheres to the "
		"high-quality standards expected for customer support.\n"
        "Verify that all parts of the customer's inquiry "
        "have been addressed "
		"thoroughly, with a helpful and friendly tone.\n"
        "Check for references and sources used to "
        " find the information, "
		"ensuring the response is well-supported and "
        "leaves no questions unanswered."
    ),
    expected_output=(
        "A final, detailed, and informative response "
        "ready to be sent to the customer.\n"
        "This response should fully address the "
        "customer's inquiry, incorporating all "
		"relevant feedback and improvements.\n"
		"Don't be too formal, we are a chill and cool company "
	    "but maintain a professional and friendly tone throughout."
    ),
    agent=support_quality_assurance_agent,
)

In [34]:
# Create the crew of agents
crew = Crew(
  agents=[support_agent, support_quality_assurance_agent],
  tasks=[inquiry_resolution, quality_assurance_review],
  verbose=2,
  memory=True
)

2024-07-30 15:35:33,881 - 139695940392768 - __init__.py-__init__:531 - WARNING: Overriding of current TracerProvider is not allowed


In [35]:
%%time
# Provide the relevant details and start the crew
inputs = {
    "customer": "Very Good AI Company",
    "person": "Solomon Kobiri",
    "inquiry": "I need help with using langchain to build agents "
               "into my web application. "
               "I am a backend developer using FastAPI "
               "Can you provide guidance?"
}
result = crew.kickoff(inputs=inputs)

 [DEBUG]: == Working Agent: Senior Support Representative
 [INFO]: == Starting Task: Very Good AI Company just reached out with a super important ask:
I need help with using langchain to build agents into my web application. I am a backend developer using FastAPI Can you provide guidance?

Solomon Kobiri from Very Good AI Company is the one that reached out. Make sure to use everything you know to provide the best support possible.You must strive to provide a complete and accurate response to the customer's inquiry.


> Entering new CrewAgentExecutor chain...
I need to find detailed information on how Langchain can be integrated into web applications, specifically using FastAPI, to assist Solomon Kobiri from Very Good AI Company.

Action: Read website content
Action Input: {"url": "https://python.langchain.com/v0.2/docs/introduction/"} 

Introduction | 🦜️🔗 LangChain
Skip to main contentLangChain 0.2 is out! Leave feedback on the v0.2 docs here. You can view the v0.1 docs here.Integrati

In [36]:
# View the output from the agents
from IPython.display import Markdown
Markdown(result)

Hello Solomon,

Thank you for reaching out for assistance on integrating LangChain into your web application using FastAPI. I'm here to provide you with detailed guidance to help you get started and make the most of LangChain’s capabilities.

LangChain is a comprehensive framework designed for developing applications powered by large language models (LLMs). It simplifies the development, productionization, and deployment stages of LLM application lifecycle. Here’s how you can leverage LangChain for your backend development with FastAPI:

1. **Development**: 
   - You can build your applications using LangChain’s open-source building blocks, components, and third-party integrations. 
   - Use LangGraph to build robust and stateful multi-actor applications. LangGraph models steps as edges and nodes in a graph, which integrates smoothly with LangChain but can also be used independently. When integrated with LangChain, LangGraph leverages its graph-based data structures to provide context and relational information, enhancing the language models' understanding and generation capabilities. Independently, LangGraph functions as a powerful tool capable of constructing and managing complex graph-based data representations, useful for applications involving complex relationships and networked data.

2. **Productionization**: 
   - LangSmith, a developer platform within LangChain, allows you to inspect, monitor, evaluate, and optimize your LLM applications continuously.

3. **Deployment**: 
   - Turn your LangGraph applications into production-ready APIs and assistants with LangGraph Cloud.
   - LangServe can be used to deploy LangChain chains as REST APIs, which you can integrate seamlessly with your FastAPI backend.

For specific steps and more detailed instructions, you might want to explore the following resources on the LangChain documentation:
- Tutorials on building simple LLM applications, chatbots, agents, and more.
- How-to guides for quick solutions to common tasks such as adding memory to chatbots, using vector stores as retrievers, etc.
- API reference for full documentation of all classes and methods in the LangChain Python packages.

Here are some links that you might find useful:
- Tutorials: [LangChain Tutorials](https://python.langchain.com/v0.2/docs/tutorials/)
- How-to Guides: [How-to Guides](https://python.langchain.com/v0.2/docs/how-to/)
- API Reference: [API Reference](https://python.langchain.com/v0.2/docs/api/)

I hope this helps you get started with your project. If you have any more questions or need further assistance, please do not hesitate to reach out. We're here to help you succeed with LangChain!

Best regards,
[Your Name]
Senior Support Representative at LangChain

# Conclusion

From the above, you can see the two agents working together to provide the best response to the customer's query.

### NB: [CrewAI](https://www.crewai.com/) has a lot more tools and functionalities that you can explore.

### Other Agentic Tools You Can Explore Are
- [Langchain Agents](https://python.langchain.com/v0.2/docs/tutorials/agents/)
- [LangGraph](https://langchain-ai.github.io/langgraph/)
- [Agency Swarm](https://vrsen.github.io/agency-swarm/)
- [Autogen](https://microsoft.github.io/autogen/)